<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/2_GPT_evaluate_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter

Esta notebook se usa para recuperar los batch de GPT y evaluarlos en relación a los datos de test

**Natalia Dedandi**





La documentación de la API se encuentra aqui:

https://platform.openai.com/docs/api-reference/introduction



In [ ]:
pip install openai

In [59]:
import json
from openai import OpenAI
import os
from google.colab import userdata
#seteo la KEY
os.environ['OPENAI_API_KEY'] = userdata.get('openIA_key')




In [60]:
#creo el cliente OPENAI con mi usuario y proyecto
client = OpenAI(
  organization='org-1uHjwiaB3OlPzoxfVzhqOSzs',
  project='proj_2fII7izwVGgYaERNNKhhMx4l',
)

pRUEBAS BINARIAS

BATCH ID Prueba 1: batch_P0l0BWckYZI3eywp5i2EsT9F

BATCH ID Prueba 2: batch_gOkX3xcR4maDJN8sxd86kvmi



In [64]:
batch_job = client.batches.retrieve('batch_gOkX3xcR4maDJN8sxd86kvmi')
status = batch_job.status
status

'in_progress'

In [ ]:
if status == "completed":

	output_file_id = batch_job.output_file_id
	# We should check that there are succesfull results
	# It could be the case that all responses are errors.
	if output_file_id:
		outputs = client.files.content(output_file_id).content
		result_file_name = "outputs.jsonl"

		with open(result_file_name, 'wb') as file:
		    file.write(outputs)

In [ ]:
import pandas as pd


data_results = []
with open(result_file_name, 'r') as file:
  for line in file:
      output = json.loads(line.strip())
      id=output["custom_id"]
      sentiment= output["response"]["body"]["choices"][0]["message"]["content"]
      data_results.append({'id': id, 'sentiment_gpt': sentiment})
      gpt_response = output["response"]["body"]["choices"][0]["message"]["content"]
      #print(gpt_response)

df_results = pd.DataFrame(data_results)

In [ ]:
df_results[0:10]

,id,sentiment_gpt
0,397703,Hateful
1,397704,No es un comentario odioso.
2,397705,El texto contiene un sentimiento de odio hacia...
3,397706,El texto tiene un tono de desdén y crítica hac...
4,397707,No hateful.
5,397708,Hateful
6,397709,Hateful
7,397710,El texto contiene un lenguaje despectivo y ofe...
8,397711,No es un mensaje de odio.
9,397712,"Lo siento, pero necesito más contexto o un tex..."


In [ ]:
df_results.to_csv('outputs_bin_1.csv')

In [ ]:
df_results.groupby(['sentiment_gpt']).size()

sentiment_gpt
Aumentos a jubilados que aportaron toda su vida laboral. Además, se implementaron mejoras en los programas de salud destinados a jubilados, lo que garantiza un acceso más fácil a servicios médicos. También se establecieron beneficios fiscales adicionales para las pensiones, aumentando así el ingreso neto de muchos jubilados. Por último, se promovieron actividades recreativas y culturales para este sector de la población, fomentando su bienestar general.    1
Claro, puedo ayudarte con eso. Por favor, proporciona el texto que deseas que analice.                                                                                                                                                                                                                                                                                                                                                                                       1
Despreciable y lleno de odio.                               

In [ ]:
from datasets import load_dataset
import pandas as pd

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")

In [ ]:
df_test = pd.DataFrame(ds1['test'])
df_test

In [ ]:
#junto los dos dataframes
df_hateful=df_test.merge(df_results, on='id', how='left')

In [ ]:
## ajustar los valores y crear una variable binaria entera 1 y 0
df_hateful['sentiment_bin'] = df_hateful['sentiment_gpt'].apply(lambda x: 1 if (x == 'odioso' or x=='hateful' or x=='odioso.') else 0)
df_hateful.groupby(['sentiment_bin']).size()